In [2]:
!nvidia-smi

Mon Nov 25 06:35:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:3B:00.0 Off |                  Off |
| 31%   40C    P0             62W /  450W |       0MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

[Errno 2] No such file or directory: '/content/'
/home/syq/Fine tuning


Cloning into 'LLaMA-Factory'...


/home/syq/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


remote: Enumerating objects: 317, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 317 (delta 78), reused 153 (delta 48), pack-reused 0 (from 0)
Receiving objects: 100% (317/317), 8.97 MiB | 6.97 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/home/syq/Fine tuning/LLaMA-Factory
assets/       evaluation/  MANIFEST.in     requirements.txt  tests/
CITATION.cff  examples/    pyproject.toml  scripts/
data/         LICENSE      README.md       setup.py
docker/       Makefile     README_zh.md    src/


/home/syq/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.1 MB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 KB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 1.5 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 1.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 902.5 kB/s eta 0:00:0000:0100:10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 774.8 kB/s eta 0:00:0000:0100:14
     ━━━━━━━━━━━━━━━━━━━

In [10]:
!pip uninstall -y vllm
!pip install llamafactory[metrics]==0.7.1
!pip install accelerate==0.30.1

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 KB 1.0 MB/s eta 0:00:0000:0100:01
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 KB 1.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.0.1
    Uninstalling accelerate-1.0.1:
      Successfully uninstalled accelerate-1.0.1


In [ ]:
# 查看llamafactory是否安装成功
!llamafactory-cli version

----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.7.1                |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------


In [11]:
# 检查torch和显卡
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.__version__

'2.3.1+cu121'

In [17]:
# 下载模型
!git clone https://www.modelscope.cn/baichuan-inc/Baichuan2-7B-Chat.git


Cloning into 'Baichuan2-7B-Chat'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 115 (delta 16), reused 14 (delta 7), pack-reused 78
Receiving objects: 100% (115/115), 468.12 KiB | 3.49 MiB/s, done.
Resolving deltas: 100% (48/48), done.
^C
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'


Exiting because of "interrupt" signal.


In [9]:
%pip install modelscope

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
!ls -lh /home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat/pytorch_model.bin

-rw-rw-r-- 1 syq syq 14G Nov 25 10:02 /home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat/pytorch_model.bin


In [ ]:
# 测试模型是否导入成功
from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# 本地模型路径
model_dir = "/home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat"

# 加载分词器
try:
    tokenizer = AutoTokenizer.from_pretrained(
        model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
    )
    print("Tokenizer loaded successfully!")
except Exception as e:
    print("Error loading tokenizer:", e)

# 加载模型
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
    )
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)

# 加载生成配置
try:
    model.generation_config = GenerationConfig.from_pretrained(model_dir)
    print("Generation config loaded successfully!")
except Exception as e:
    print("Error loading generation config:", e)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Tokenizer loaded successfully!
Model loaded successfully!
Generation config loaded successfully!


In [ ]:
!CUDA_VISIBLE_DEVICES=0 llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path /home/syq/fine_tuning/test1/LLaMA-Factory/Baichuan2-7B-Chat \
    --dataset train \
    --dataset_dir /home/syq/fine_tuning/test1/LLaMA-Factory/data \
    --template baichuan2 \
    --finetuning_type lora \
    --output_dir ./saves/baichuan/lora/sft \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 1024 \
    --preprocessing_num_workers 16 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 100 \
    --warmup_steps 500 \
    --save_steps 1000 \
    --eval_steps 1000 \
    --evaluation_strategy steps \
    --load_best_model_at_end \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --max_samples 1000 \
    --val_size 0.1 \
    --plot_loss \
    --fp16

/home/syq/.local/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
11/26/2024 04:56:50 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2209] 2024-11-26 04:56:50,132 >> loading file tokenizer.model
[INFO|tokenization_utils_base.py:2209] 2024-11-26 04:56:50,132 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2209] 2024-11-26 04:56:50,132 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2209] 2024-11-26 04:56:50,132 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2209] 2024-11-26 04:56:50,132 >> loading file tokenizer.json
11/26/2024 04:56:50 - INFO - llamafactory.data.loader - Loading dataset train.json...
Converting format of dataset (n